In [1]:
#Import packages
import boto3
import json
import numpy as np
from numpy import argmax
import pyspark.sql.functions as F
import snowflake.connector
import pandas as pd
import spacy
import matplotlib.pyplot as plt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from urllib.parse import urlparse
from langdetect import detect
from textblob import TextBlob
#RegEx
import regex as re

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1616589606726_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
#textblob package

from textblob import TextBlob

text = '''
excellent
'''

blob = TextBlob(text)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
blob.tags

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('excellent', 'NN')]

In [16]:
#Believe these fail bc it uses Google Translate and either it's not connecting right or Google is rejecting bc of ip address or something else
blob.translate(to="es")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

HTTP Error 429: Too Many Requests
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/textblob/blob.py", line 547, in translate
    from_lang=from_lang, to_lang=to))
  File "/usr/local/lib/python3.7/site-packages/textblob/translate.py", line 54, in translate
    response = self._request(url, host=host, type_=type_, data=data)
  File "/usr/local/lib/python3.7/site-packages/textblob/translate.py", line 92, in _request
    resp = request.urlopen(req)
  File "/usr/lib64/python3.7/urllib/request.py", line 222, in urlopen
    return opener.open(url, data, timeout)
  File "/usr/lib64/python3.7/urllib/request.py", line 531, in open
    response = meth(req, response)
  File "/usr/lib64/python3.7/urllib/request.py", line 641, in http_response
    'http', request, response, code, msg, hdrs)
  File "/usr/lib64/python3.7/urllib/request.py", line 563, in error
    result = self._call_chain(*args)
  File "/usr/lib64/python3.7/urllib/request.py", line 503, in _call_chain


In [17]:
blob.detect_language()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

HTTP Error 429: Too Many Requests
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/textblob/blob.py", line 568, in detect_language
    return self.translator.detect(self.raw)
  File "/usr/local/lib/python3.7/site-packages/textblob/translate.py", line 72, in detect
    response = self._request(url, host=host, type_=type_, data=data)
  File "/usr/local/lib/python3.7/site-packages/textblob/translate.py", line 92, in _request
    resp = request.urlopen(req)
  File "/usr/lib64/python3.7/urllib/request.py", line 222, in urlopen
    return opener.open(url, data, timeout)
  File "/usr/lib64/python3.7/urllib/request.py", line 531, in open
    response = meth(req, response)
  File "/usr/lib64/python3.7/urllib/request.py", line 641, in http_response
    'http', request, response, code, msg, hdrs)
  File "/usr/lib64/python3.7/urllib/request.py", line 563, in error
    result = self._call_chain(*args)
  File "/usr/lib64/python3.7/urllib/request.py", line 503, in _ca

In [6]:
#guess_language package


from guess_language import guess_language
#guess_language("Ces eaux regorgent de renégats et de voleurs.")

#Need separate thing if less than ~20 characters
#https://pypi.org/project/guess_language-spirit/
#Need PyEnchant (spell checker with dictionaries) - not sure how to use this for detecting langs
#and how many dictionaries needed bc we support LOTS of different languages
guess_language("good good good good good")
#guess_language("i like this it is awesome")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'cy'

In [7]:
#polyglot package

#Errors
#Need to also install
#sudo apt-get install python-numpy libicu-dev
from polyglot.detect import Detector

arabic_text = u"""
أفاد مصدر امني في قيادة عمليات صلاح الدين في العراق بأن " القوات الامنية تتوقف لليوم
الثالث على التوالي عن التقدم الى داخل مدينة تكريت بسبب
انتشار قناصي التنظيم الذي يطلق على نفسه اسم "الدولة الاسلامية" والعبوات الناسفة
والمنازل المفخخة والانتحاريين، فضلا عن ان القوات الامنية تنتظر وصول تعزيزات اضافية ".
"""

detector = Detector(arabic_text)
print(detector.language)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

No module named 'icu'
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/polyglot/detect/__init__.py", line 1, in <module>
    from .base import Detector, Language
  File "/usr/local/lib/python3.7/site-packages/polyglot/detect/base.py", line 11, in <module>
    from icu import Locale
ModuleNotFoundError: No module named 'icu'



In [19]:
#googletrans


#Error - probably bc calls Google API and I think this notebook through EMR cluster can't call the same way my local machine might
from googletrans import Translator
t = Translator().detect("hello world!")
t.lang

#t.confidence

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'NoneType' object has no attribute 'group'
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/googletrans/client.py", line 255, in detect
    data = self._translate(text, 'en', 'auto', kwargs)
  File "/usr/local/lib/python3.7/site-packages/googletrans/client.py", line 78, in _translate
    token = self.token_acquirer.do(text)
  File "/usr/local/lib/python3.7/site-packages/googletrans/gtoken.py", line 194, in do
    self._update()
  File "/usr/local/lib/python3.7/site-packages/googletrans/gtoken.py", line 62, in _update
    code = self.RE_TKK.search(r.text).group(1).replace('var ', '')
AttributeError: 'NoneType' object has no attribute 'group'



In [20]:
from googletrans import Translator
translator = Translator(service_urls=['translate.googleapis.com'])
translator.translate("Der Himmel ist blau und ich mag Bananen", dest='en')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'NoneType' object has no attribute 'group'
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/googletrans/client.py", line 182, in translate
    data = self._translate(text, dest, src, kwargs)
  File "/usr/local/lib/python3.7/site-packages/googletrans/client.py", line 78, in _translate
    token = self.token_acquirer.do(text)
  File "/usr/local/lib/python3.7/site-packages/googletrans/gtoken.py", line 194, in do
    self._update()
  File "/usr/local/lib/python3.7/site-packages/googletrans/gtoken.py", line 62, in _update
    code = self.RE_TKK.search(r.text).group(1).replace('var ', '')
AttributeError: 'NoneType' object has no attribute 'group'



In [6]:
from google_trans_new import google_translator  
  
detector = google_translator()  
detect_result = detector.detect('this is excelent')

print(detect_result[0])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

en

In [9]:
import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector

def get_lang_detector(nlp, name):
    return LanguageDetector()

nlp = spacy.load("en_core_web_sm")
Language.factory("language_detector", func=get_lang_detector)
nlp.add_pipe('language_detector', last=True)
text = 'This is an english text.'
doc = nlp(text)
print(doc._.language)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'language': 'en', 'score': 0.9999977076591322}

In [10]:
lang = doc._.language
lang['language']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'en'

In [11]:
import pycld2

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

No module named 'pycld2'
Traceback (most recent call last):
ModuleNotFoundError: No module named 'pycld2'



In [12]:
import pycld2 as cld2

isReliable, textBytesFound, details = cld2.detect(
    "а неправильный формат идентификатора дн назад"
)

print(isReliable)
# True
details[0]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

No module named 'pycld2'
Traceback (most recent call last):
ModuleNotFoundError: No module named 'pycld2'



In [45]:
#Use to check individual words if English or not
#Error - need to install other libraries but not sure how to do this on EMR cluster

import enchant
d = enchant.Dict("en_US")
d.check("Hello")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

The 'enchant' C library was not found and maybe needs to be installed.
See  https://pyenchant.github.io/pyenchant/install.html
for details

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/enchant/__init__.py", line 81, in <module>
    from enchant import _enchant as _e
  File "/usr/local/lib/python3.7/site-packages/enchant/_enchant.py", line 157, in <module>
    raise ImportError(msg)
ImportError: The 'enchant' C library was not found and maybe needs to be installed.
See  https://pyenchant.github.io/pyenchant/install.html
for details




In [14]:
from nltk.corpus import words
"pasta" in words.words()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

False

In [3]:
import boto3
import json

comprehend = boto3.client(service_name='comprehend', region_name='us-west-2')
text = "Las flores son muy bonita"

print('Calling DetectDominantLanguage')
print(json.dumps(comprehend.detect_dominant_language(Text = text), sort_keys=True, indent=4))
print("End of DetectDominantLanguage\n")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Calling DetectDominantLanguage
{
    "Languages": [
        {
            "LanguageCode": "es",
            "Score": 0.854913055896759
        },
        {
            "LanguageCode": "en",
            "Score": 0.14185743033885956
        }
    ],
    "ResponseMetadata": {
        "HTTPHeaders": {
            "content-length": "113",
            "content-type": "application/x-amz-json-1.1",
            "date": "Tue, 06 Apr 2021 16:01:52 GMT",
            "x-amzn-requestid": "8ccbd6f6-56b6-4a2c-a5a2-8e6dd14ceb6b"
        },
        "HTTPStatusCode": 200,
        "RequestId": "8ccbd6f6-56b6-4a2c-a5a2-8e6dd14ceb6b",
        "RetryAttempts": 0
    }
}
End of DetectDominantLanguage

In [15]:
sm_client = boto3.client('secretsmanager',region_name='us-west-2')
secret_value = sm_client.get_secret_value(SecretId = 'emr_spark_qa')
username = json.loads(secret_value['SecretString'])['emr_spark_username']
password = json.loads(secret_value['SecretString'])['emr_spark_password']
#print(username)
#print(password)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
#PYTHON connection info
conn = snowflake.connector.connect(
    account='powerreviews',
    database='analytics_prod',
    warehouse='LOWER_ENV_WAREHOUSE',
    user=username,
    password=password
    )


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
#Query some reviews
query_1 = "select distinct r.ugc_id, r.review_headline, r.review_comments, substr(r.locale,1,2) as lang, r.review_length, \
case when contains(r.observations, 'FL')  then 1 else 0 end as FL \
from analytics.review as r \
join analytics.merchant_group as mg \
on r.client_id = mg.client_id \
where contains(r.observations, 'PM') \
and not contains(r.observations, 'IA') and not contains(r.observations, 'IR') \
and mg.usage_type != 'Test' \
and r.created_date > '2020-01-01' \
limit 10000 \
"

#193267606, 324912854
#and r.created_date > '2020-01-01' \

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
reviews = pd.read_sql(query_1, conn)
reviews.head()
reviews.info()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   UGC_ID           100 non-null    int64 
 1   REVIEW_HEADLINE  100 non-null    object
 2   REVIEW_COMMENTS  100 non-null    object
 3   LANG             100 non-null    object
 4   REVIEW_LENGTH    100 non-null    int64 
 5   FL               100 non-null    int64 
dtypes: int64(3), object(3)
memory usage: 4.8+ KB

In [19]:
#Need to add "word" in front bc otherwise reviews with just emojis, symbols, #s, etc will not work in langdetect package
#Throws this error if no words: "langdetect.lang_detect_exception.LangDetectException: No features in text."
reviews['ALL_TEXT'] = reviews['REVIEW_HEADLINE'] + ' ' + reviews['REVIEW_COMMENTS']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:


from guess_language import guess_language
#guess_language("Ces eaux regorgent de renégats et de voleurs.")

#Need separate thing if less than ~20 characters
#https://pypi.org/project/guess_language-spirit/
#Need PyEnchant (spell checker with dictionaries) - not sure how to use this for detecting langs
#and how many dictionaries needed bc we support LOTS of different languages

#guess_language("good good good good good")
reviews['ALL_TEXT']=reviews['ALL_TEXT'].apply(str)
reviews['PRED_LANG'] = reviews['ALL_TEXT'].apply(guess_language)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
#Check if 1st and 2nd words are in English dictionary set
reviews['FIRST_WORD'] = reviews['ALL_TEXT'].str.split(' ').str[0]
reviews['SECOND_WORD'] = reviews['ALL_TEXT'].str.split(' ').str[1]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
reviews.info()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   UGC_ID           100 non-null    int64 
 1   REVIEW_HEADLINE  100 non-null    object
 2   REVIEW_COMMENTS  100 non-null    object
 3   LANG             100 non-null    object
 4   REVIEW_LENGTH    100 non-null    int64 
 5   FL               100 non-null    int64 
 6   ALL_TEXT         100 non-null    object
 7   PRED_LANG        100 non-null    object
 8   FIRST_WORD       100 non-null    object
 9   SECOND_WORD      100 non-null    object
dtypes: int64(3), object(7)
memory usage: 7.9+ KB

In [23]:
from nltk.corpus import words
wordlist = words.words()
#reviews['FIRST_WORD']=reviews['FIRST_WORD'].apply(str)
reviews['test'] = reviews['FIRST_WORD'].isin(wordlist)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
wordnet.synsets("Leather")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Synset('leather.n.01'), Synset('leather.v.01')]

In [35]:
"should" in words.words()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

True

In [36]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

reviews.head(25)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

       UGC_ID                             REVIEW_HEADLINE  \
0   282060739  We knew of SpringFree fro                    
1   269284201  I would purchase this again.                 
2   318907826  Love it!!                                    
3   318907879  Nuun Immunity                                
4   336905806  Leather balm                                 
5   336928917  As always, awesome socks!                    
6   336906245  Saved my Money.                              
7   298126327  Beautiful magazine, just comes very late     
8   336934937  very big                                     
9   336916657  The shirts are comfortable, stylish          
10  336924853  ...                                          
11  298126961  OMG                                          
12  298126330  Simply Beautiful                             
13  269262930  Anyone have the instructions?                
14  334836029  Great Product - however ...                  
15  334833221  My hair f

In [37]:
reviews['model_FL'] = 0

reviews.loc[reviews['LANG'] != reviews['PRED_LANG'] , 'model_FL'] = 1

reviews.loc[reviews['REVIEW_LENGTH'] < 40 , 'model_FL'] = 0



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
FP = reviews[(reviews['FL'] == 0) & (reviews['model_FL'] == 1)]
FN = reviews[(reviews['FL'] == 1) & (reviews['model_FL'] == 0)]
TP = reviews[(reviews['FL'] == 1) & (reviews['model_FL'] == 1)]
TN = reviews[(reviews['FL'] == 0) & (reviews['model_FL'] == 0)]


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:

FP['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [40]:

FN['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [41]:

TP['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [42]:
TN['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

100

In [84]:
FP.head(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

         UGC_ID            REVIEW_HEADLINE  \
991   285823949  Great purchase              
1758  330939553  Mediterranean Honeysuckle   
1786  330944794  Perfume                     
1795  330946020  Jsjsjs                      
1805  330937618  Extremadamente bueno        
1983  330838063  Undisclosed                 
2138  330891348  Lightweight                 
2298  330888191  Fab product                 
2460  330898234  Great lightweight jacket    
2696  330848993  Good stron                  

                                                                                            REVIEW_COMMENTS  \
991   My new favorite flip flop.  My go to shoe.  Want to get more.                                           
1758  Mediterranean Honeysuckle smells really good cant wait to get a bottle                                  
1786  Smells good but I prefer Mediterranean Honeysuckle in Bloom.                                            
1795  Jsjsjs feet ddfvvbh dsfv fwdgh dargfs cwwrg

In [19]:
detect('good good stuff')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'cy'

In [28]:
FP.info()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6456 entries, 638 to 8681798
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   UGC_ID               6456 non-null   int64 
 1   NAME                 6456 non-null   object
 2   REVIEW_HEADLINE      6456 non-null   object
 3   REVIEW_COMMENTS      6456 non-null   object
 4   CR                   6456 non-null   int64 
 5   ALL_TEXT             6456 non-null   object
 6   NAME_1ST_PART_RAW    6456 non-null   object
 7   NAME_1ST_PART        6456 non-null   object
 8   ALL_TEXT_2           6456 non-null   object
 9   ALL_TEXT_3           6456 non-null   object
 10  ALL_TEXT_4           6456 non-null   object
 11  ALL_TEXT_5           6456 non-null   object
 12  ALL_TEXT_6           6456 non-null   object
 13  ALL_TEXT_7           6456 non-null   object
 14  regex_retailer       6456 non-null   object
 15  retailer_not_client  6456 non-null   bool  
 16  m

In [29]:
from pyspark.sql.types import *

mySchema = StructType([StructField("UGC_ID", StringType(), True)\
,StructField("NAME", StringType(), True)\
,StructField("REVIEW_HEADLINE", StringType(), True)\
,StructField("REVIEW_COMMENTS", StringType(), True)\
,StructField("CR", IntegerType(), True)\
,StructField("ALL_TEXT", StringType(), True)\
,StructField("NAME_1ST_PART_RAW", StringType(), True)\
,StructField("NAME_1ST_PART", StringType(), True)\
,StructField("ALL_TEXT_2", StringType(), True)\
,StructField("ALL_TEXT_3", StringType(), True)\
,StructField("ALL_TEXT_4", StringType(), True)\
,StructField("ALL_TEXT_5", StringType(), True)\
,StructField("ALL_TEXT_6", StringType(), True)\
,StructField("ALL_TEXT_7", StringType(), True)\
,StructField("regex_retailer", StringType(), True)\
,StructField("retailer_not_client", StringType(), True)\
,StructField("model_CR", IntegerType(), True)])



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"

sfOptions = {
  'sfURL' : 'powerreviews.snowflakecomputing.com',
  'sfAccount' : 'powerreviews',
  'sfUser' : username,
  'sfPassword' : password,
  'sfDatabase' : 'ANALYTICS_WORKSHOP',
  'sfSchema' : 'DATA_SCIENCE',
  'sfWarehouse' : 'LOWER_ENV_WAREHOUSE'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
#Turn into spark dataframe in order to write to Snowflake
#Couldn't get write_pandas() to work to write directly from pandas to Snowflake
FP = spark.createDataFrame(FP, schema=mySchema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
FP.write.format(SNOWFLAKE_SOURCE_NAME).options(**sfOptions) \
    .option("dbtable","CR_FP_test4").mode("append").save()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [109]:
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TST_FP_TEST3")
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TST_FN_TEST3")
#conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TEST_NLP_SPARK6")
#conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TEST_NLP_SWAP_TOKENS")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…